In [1]:
import pandas as pd
import json
import numpy as np
import os
from tqdm import tqdm
from tabulate import tabulate
import random
import matplotlib.pyplot as plt
from urllib.request import urlopen
from Author_Stats import Author_Stats

C:\Users\ACER\AppData\Local\Temp\ipykernel_4480\1450419276.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Useful Functions

In [92]:
# Python3 program two find number of
# days between two given dates

# A date has day 'd', month 'm' and year 'y'


class Date:
	def __init__(self, d, m, y):
		self.d = d
		self.m = m
		self.y = y


# To store number of days in all months from
# January to Dec.
monthDays = [31, 28, 31, 30, 31, 30,
			31, 31, 30, 31, 30, 31]

# This function counts number of leap years
# before the given date


def countLeapYears(d):

	years = d.y

	# Check if the current year needs to be considered
	# for the count of leap years or not
	if (d.m <= 2):
		years -= 1

	# An year is a leap year if it is a multiple of 4,
	# multiple of 400 and not a multiple of 100.
	return int(years / 4) - int(years / 100) + int(years / 400)


# This function returns number of days between two
# given dates
def getDifference(dt1, dt2):

	# COUNT TOTAL NUMBER OF DAYS BEFORE FIRST DATE 'dt1'

	# initialize count using years and day
	n1 = dt1.y * 365 + dt1.d

	# Add days for months in given date
	for i in range(0, dt1.m - 1):
		n1 += monthDays[i]

	# Since every leap year is of 366 days,
	# Add a day for every leap year
	n1 += countLeapYears(dt1)

	# SIMILARLY, COUNT TOTAL NUMBER OF DAYS BEFORE 'dt2'

	n2 = dt2.y * 365 + dt2.d
	for i in range(0, dt2.m - 1):
		n2 += monthDays[i]
	n2 += countLeapYears(dt2)

	# return difference between two counts
	return (n2 - n1)


# Driver program
dt1 = Date(13, 12, 2018)
dt2 = Date(25, 2, 2019)

print(getDifference(dt1, dt2), "days")


74 days


In [98]:
def Date_Diff(date_1,date_2):
    y1,y2 = int(date_1[:4]),int(date_2[:4])
    m1,m2 = int(date_1[5:7]),int(date_2[5:7])
    d1,d2 = int(date_1[8:10]),int(date_2[8:10])
    return getDifference(Date(d1,m1,y1),Date(d2,m2,y2))

In [175]:
def average_gap_finder(Institutes,direct=False):
    Dates = []
    if direct:
        for paper in papers:
            Dates.append(paper["publication_date"])
    else:
        for institute in Institutes:
            Dates.append(institute[3])
    Gaps = [Date_Diff(Dates[i],Dates[i+1]) for i in range(len(Dates)-1) ]
    return np.mean(Gaps)

# Tracking Career

In [3]:
PATH = "C:\\Users\\ACER\\Desktop\\INTERNSHIP\\Career\\Stats.txt"
f = open(PATH, encoding='utf8')
Stats = json.load(f)
f.close()
df = pd.read_csv("C:\\Users\\ACER\\Desktop\\INTERNSHIP\\Career\\Author_Details.csv")

In [4]:
ADB = Author_Stats(df=df,Stats=Stats)

In [181]:
Institution_Heirarchy = {'education': 0,'facility': 1,'government':2,'healthcare':3,'company':4,'nonprofit':5}
def Describe_Career(Open_Alex_ID,Author_Data_Base,return_=False,filter_by_gap=True,filter_coefficient= 3):
    
    papers = Author_Data_Base.Get_Works(Open_Alex_ID = Open_Alex_ID,return_=True,Show_Paper_names=False)
    #Affiliations = {paper['publication_year']: [] for paper in papers}
    Institute_list_initial = {"A": [],"B": [] }
    #Institute_list = set({})
    #AVG_GAP = average_gap_finder(Papers=papers)
    Common = []
    
    
    for i in range(len(papers)):
        for author in papers[i]['authorships']:
            if author['author']['id'] == Open_Alex_ID:
                for institute in author['institutions']:
                    #Affiliations[papers[i]['publication_year']].append((institute['display_name'],institute['country_code'],institute['type'],papers[i]['publication_date'],i))
                    #Institute_list.add(institute['display_name'])
                    if i < 1:
                        Institute_list_initial["A"].append([institute['display_name'],institute['country_code'],institute['type'],papers[i]['publication_date']])
                    else:
                        Institute_list_initial["B"].append([institute['display_name'],institute['country_code'],institute['type'],papers[i]['publication_date']])
        #Flag = True
        if i>=1:
            for j in range(len(Institute_list_initial["B"])):
                for k in range(len(Institute_list_initial["A"])):
                    if Institute_list_initial["B"][j][0] == Institute_list_initial["A"][k][0]:
                        Flag = True
                        for institute in Common:
                            if institute[0] == Institute_list_initial["B"][j][0]:
                                Flag = False
                        if Flag:
                            Common.append(Institute_list_initial["B"][j])
            Institute_list_initial['A'] = Institute_list_initial['B'] 
            Institute_list_initial['B'] = []
        #print(Common)       
    #Institute_list = list(Institute_list)               
    #return Affiliations,Institute_list,Common
    AFFILIATIONS = []
    if filter_by_gap:
        AVG_GAP = average_gap_finder(Common)
        for i in range(len(Common)-1):
            if Date_Diff(Common[i][3],Common[i+1][3]) <= filter_coefficient*AVG_GAP or Date_Diff(Common[i][3],Common[i+1][3]) <= 5*365:
                AFFILIATIONS.append(Common[i])
        AFFILIATIONS.append(Common[i+1])
    else:
        AFFILIATIONS = Common
    if return_:
        return AFFILIATIONS
    print(f"No of papers visited: {len(papers)}")
    print(tabulate(pd.DataFrame(AFFILIATIONS)))
#aff = Describe_Career(Open_Alex_ID= df['id'][0],Author_Data_Base= ADB)



In [183]:
# Highest H-Index is 90
# Filtering for scientists who have H-Index above 70

i = random.choice(np.array(df['id'][df['H-index']>=70]))
#print(f"ID : {i}")
Describe_Career(Open_Alex_ID = i,Author_Data_Base=ADB,filter_by_gap=False)
Describe_Career(Open_Alex_ID = i,Author_Data_Base=ADB,filter_coefficient=3)
#Describe_Career(Open_Alex_ID = df['id'][i],Author_Data_Base=ADB,filter_coefficient=2)

Index : https://openalex.org/A5070986885
Name : John H. R. Maunsell
H-Index : 78
Open ALex ID: https://openalex.org/A5070986885
------------------------------------------------------------------------------------------
No of papers visited: 188
--  -----------------------------------------------------  --  ----------  ----------
 0  Andover Eye Associates                                 US  nonprofit   1934-04-01
 1  University of Iowa                                     US  education   1934-04-01
 2  California Institute of Technology                     US  education   1980-12-01
 3  University of Rochester                                US  education   1989-10-01
 4  University of Rochester Medical Center                 US  healthcare  1991-11-01
 5  Baylor College of Medicine                             US  education   1995-11-03
 6  Howard Hughes Medical Institute                        US  nonprofit   1999-01-01
 7  National Institute for Occupational Safety and Health  US  faci